In [ ]:
import datetime

import pandas as pd
import numpy as np

import yfinance as yf

import altair as alt

In [ ]:
# Parameters

stock_ticker = "HSY"

# check nasdaq dividend history to find a schedule
dividend_schedule = 'quarterly'

today = datetime.date.today()
year = today.year
starting_year = year - 20

In [ ]:
# Fetch/Make Calls to y-finance

stock = yf.Ticker(stock_ticker)

dividends = stock.dividends.loc[f'{starting_year}-01-01':].to_frame().reset_index()
stock_price_history = stock.history(period="240mo").reset_index() # 20 years

In [ ]:
## Pre-processing:


# Estimate current dividend year
if dividend_schedule == 'quarterly':
    quarters = len(dividends[dividends.Date.dt.year == year])
    last_date = dividends.iloc[len(dividends)-1].Date
    last_indx = len(dividends)-1

    if quarters < 4:
        avg_dividends = dividends[dividends.Date.dt.year == year].Dividends.mean()

    for i in range(1,4-quarters+1):
        dividends.loc[last_indx+i] = [dividends.iloc[last_indx].Date + pd.DateOffset(months=3*i), avg_dividends]


dividends_by_year = dividends.groupby(dividends.Date.dt.year)['Dividends'].sum()
stock_price_by_year = stock_price_history.groupby(stock_price_history.Date.dt.year)['Close'].mean()

dividend_yield_by_year =  dividends_by_year / stock_price_by_year * 100
dividend_yield_by_year.rename('Dividend Yield (%)', inplace=True)

df=pd.concat([dividends_by_year,stock_price_by_year,dividend_yield_by_year],axis=1)

# Stock prices by month
stock_price_by_month = stock_price_history.groupby([stock_price_history.Date.dt.year, stock_price_history.Date.dt.month])['Close'].mean().to_frame()

payload = []
for year, month in stock_price_by_month.index:
    year_month_date = f"{year}-{month}-01"
    payload.append(year_month_date)


stock_price_by_month['Date'] = pd.to_datetime(payload)
stock_price_by_month.reset_index(drop = True, inplace = True)


# Calculate Dividend Growth pct:

growth_pcts = []
for i in range(0,len(df.Dividends)-1):

    growth_pct = round(df.Dividends.iloc[i+1]/df.Dividends.iloc[i] - 1,3)*100

    growth_pcts.append(growth_pct)

growth_pcts.insert(0, np.nan)
df['Dividend Growth % year-over-year'] = growth_pcts
df.reset_index(inplace = True)

# remove current year because incomplete data (unless able to incorporate Wall Street estimates)
#df = df.iloc[:-1,:]

## Merging
"""
combined = pd.merge(dividends.assign(grouper=dividends['Date'].dt.to_period('Y')),
               stock_price_history.assign(grouper=stock_price_history['Date'].dt.to_period('Y')),
               how='left', on='grouper')

"""

"\ncombined = pd.merge(dividends.assign(grouper=dividends['Date'].dt.to_period('Y')),\n               stock_price_history.assign(grouper=stock_price_history['Date'].dt.to_period('Y')),\n               how='left', on='grouper')\n\n"

In [ ]:
dividends.tail()

,Date,Dividends
79,2022-11-17 00:00:00-05:00,1.036
80,2023-02-16 00:00:00-05:00,1.036
81,2023-05-18 00:00:00-04:00,1.036
82,2023-08-17 00:00:00-04:00,1.192
83,2023-11-17 00:00:00-05:00,1.088


In [ ]:
df.tail()

,Date,Dividends,Close,Dividend Yield (%),Dividend Growth % year-over-year
16,2019,2.990,123.410700,2.422805,8.5
17,2020,3.154,134.238593,2.349548,5.5
18,2021,3.410,162.568671,2.097575,8.1
19,2022,3.874,214.122651,1.809243,13.6
20,2023,4.352,236.292341,1.841786,12.3


In [ ]:
dividend_rate_chart = alt.Chart(dividends).mark_line(color = 'gray').encode(
    x= "Date",
    y= alt.Y('Dividends:Q', title = 'Dividend Rate ($)')
).properties(
    title = f"Dividend Rate / Share Price - {stock_ticker}"
)

share_price_chart = alt.Chart(stock_price_by_month).mark_line(color = 'black').encode(
    x= "Date:T",
    y= alt.Y('Close:Q', title = f"{stock_ticker} Share Price ($)")
)

In [ ]:
alt.layer(share_price_chart, dividend_rate_chart).resolve_scale(
    y='independent'
).configure_axisLeft(titleColor='black').configure_axisRight(titleColor='gray')

alt.LayerChart(...)

In [ ]:
dividend_yield_line_chart = alt.Chart(df).mark_line(color = 'black').encode(
    x= alt.X("Date:O"),
    y= alt.Y('Dividend Yield (%):Q', title = 'Dividend Yield (%')
).properties(
    title = f"Dividend Analysis: {stock_ticker}"
)

dividend_growth_bar_chart = alt.Chart(df).mark_bar(opacity=0.7, color='#57A44C').encode(
    x= alt.X('Date:O'),
    y=alt.Y('Dividend Growth % year-over-year:Q', title = f'{stock_ticker} Dividend Growth, Year-over-Year(%)', 
        ),

)

In [ ]:
alt.layer(dividend_yield_line_chart, dividend_growth_bar_chart ).resolve_scale(
    y='independent'
).configure_axisLeft(titleColor='black').configure_axisRight(titleColor='#57A44C')

alt.LayerChart(...)

In [ ]:
div_growth_20_years = np.mean(df['Dividend Growth % year-over-year'])
yield_20_years = np.mean(df['Dividend Yield (%)'])

div_growth_10_years = np.mean(df['Dividend Growth % year-over-year'].iloc[9:])
yield_10_years = np.mean(df['Dividend Yield (%)'].iloc[9:])

div_growth_5_years = np.mean(df['Dividend Growth % year-over-year'].iloc[-5:])
yield_5_years = np.mean(df['Dividend Yield (%)'].iloc[-5:])

In [ ]:
def div_average_text():
    print("Averages           | Div Growth %  | Yield" )
    if len(df.Date) > 19:
        print(f"{df.Date.iloc[0]}-{df.Date.iloc[-1]} (20 yrs) | {div_growth_20_years:.2f} | {yield_20_years:.2f}")
        print(f"{df.Date.iloc[9]}-{df.Date.iloc[-1]} (10 yrs) | {div_growth_10_years:.2f} | {yield_10_years:.2f}")
        print(f"{df.Date.iloc[15]}-{df.Date.iloc[-1]} (5 yrs)  | {div_growth_5_years:.2f} | {yield_5_years:.2f}")

    elif len(df.Date) > 9:
        print(f"{df.Date.iloc[-10]}-{df.Date.iloc[-1]} (10 yrs) | {div_growth_10_years:.2f} | {yield_10_years:.2f}")
        print(f"{df.Date.iloc[-5]}-{df.Date.iloc[-1]} (5 yrs)  | {div_growth_5_years:.2f} | {yield_5_years:.2f}")

    elif len(df.Date) > 4:
        print(f"{df.Date.iloc[-5]}-{df.Date[-1]} (5 yrs)  | {div_growth_5_years:.2f} | {yield_5_years:.2f}")


    else:
        print('Historical Data less than 5 years')    


In [ ]:
div_average_text()

Averages           | Div Growth %  | Yield
2003-2023 (20 yrs) | 9.46 | 2.90
2012-2023 (10 yrs) | 10.08 | 2.51
2018-2023 (5 yrs)  | 9.60 | 2.10


In [ ]:
df.head(25)

,Date,Dividends,Close,Dividend Yield (%),Dividend Growth % year-over-year
0,2003,0.723,24.315214,2.973447,NaN
1,2004,0.835,29.316865,2.848190,15.5
2,2005,0.930,38.908824,2.390203,11.4
3,2006,1.030,35.122461,2.932596,10.8
4,2007,1.136,32.675610,3.476599,10.3
5,2008,1.192,25.520694,4.670719,4.9
6,2009,1.192,26.430240,4.509985,0.0
7,2010,1.280,33.867446,3.779441,7.4
8,2011,1.380,42.463751,3.249831,7.8
9,2012,1.560,52.851949,2.951641,13.0


## Dividend Drill Return Model

#### Last Year's DDRM

In [ ]:
# Past Year's DDRM

date = year - 1

dividend_rate = df[df['Date'] == date].Dividends.values[0]
share_price = df[df['Date'] == date].Close.values[0]
last_year_yield = df[df['Date'] == date].values[0][3]/100

last_year_financials = stock.financials.iloc[:,0]

net_income = getattr(last_year_financials, 'Net Income')

shares = getattr(last_year_financials, 'Basic Average Shares')

EPS = getattr(last_year_financials, 'Basic EPS')

ROE = net_income / getattr(stock.balance_sheet.iloc[:,0], 'Stockholders Equity')

PE = share_price / EPS

# Growth Rate (requires some thought and can be adjusted, looking at past growth rate can help.)
GR_estimate = 0.05

cost_of_growth = GR_estimate / ROE * EPS

funding_gap = EPS  - dividend_rate - cost_of_growth

share_change = funding_gap / share_price

total_dividend_growth = GR_estimate + share_change

projected_total_return = total_dividend_growth + last_year_yield

In [ ]:
print(f'{date} - {stock_ticker}')

print(f'Dividend Yield: {last_year_yield*100:.2f}')
print(f'P/E Ratio : {PE}')

print("---------------------------------------")

print(f"Dividend Rate ($): {dividend_rate:.2f}")
print(f"Divided by Share Price ($) {share_price:.2f}")
print(f"Current Yield (%): {last_year_yield*100:.2f}")

print("---------------------------------------")

print(f"Core Growth Estimate (%): {GR_estimate * 100:.2f}")
print(f"Divided by: Return on Equity (%): {ROE * 100:.2f}")
print(f"Multipled by: Earnings per Share ($): {EPS:.2f}")
print(f"Cost of Growth ($): {cost_of_growth:.2f}")

print("---------------------------------------")
print(f"Earnings per Share ($): {EPS:.2f}")
print(f"Minus: Dividend: {dividend_rate:.2f}")
print(f"Minute: Cost of Growth ($): {cost_of_growth:.2f}")
print(f"Funding Gap ($): {funding_gap:.2f}")

print("---------------------------------------")
print(f"Funding Gap ($): {funding_gap:.2f}")
print(f"Divided by: Share Price ($): {EPS:.2f}")
print(f"Share Change (%): {share_change *100:.2f}")

print("---------------------------------------")
print(f"Core Growth Estimate (%): {GR_estimate*100:.2f}")
print(f"Plus: Share Change (%): {share_change*100:.2f}")
print(f"Total Dividend Growth (%): {total_dividend_growth*100:.2f}")


print("---------------------------------------")
print(f"Plus: Dividend Yield (%): {last_year_yield*100:.2f}")
print(f"Projected Total Return (%): {projected_total_return*100:.2f}")


2022 - HSY
Dividend Yield: 1.81
P/E Ratio : 26.048984284087133
---------------------------------------
Dividend Rate ($): 3.87
Divided by Share Price ($) 214.12
Current Yield (%): 1.81
---------------------------------------
Core Growth Estimate (%): 5.00
Divided by: Return on Equity (%): 49.85
Multipled by: Earnings per Share ($): 8.22
Cost of Growth ($): 0.82
---------------------------------------
Earnings per Share ($): 8.22
Minus: Dividend: 3.87
Minute: Cost of Growth ($): 0.82
Funding Gap ($): 3.52
---------------------------------------
Funding Gap ($): 3.52
Divided by: Share Price ($): 8.22
Share Change (%): 1.64
---------------------------------------
Core Growth Estimate (%): 5.00
Plus: Share Change (%): 1.64
Total Dividend Growth (%): 6.64
---------------------------------------
Plus: Dividend Yield (%): 1.81
Projected Total Return (%): 8.45


#### DDRM (ttm)

In [ ]:
# See chapter 7 in Dividend Playbook

date = year

dividend_rate = stock.info['dividendRate']
share_price = stock.info['currentPrice']
current_yield = stock.info['dividendYield']

# Net Income
net_income = stock.info['netIncomeToCommon']

# number of shares out there.
shares_outstanding = stock.info['sharesOutstanding']

# Earnings Per Share (accounts for dividend)
EPS = net_income / shares_outstanding

#E/P Ratio
pegRatio_yf = (stock.info['trailingPE'] + stock.info['forwardPE'])/2

# Return on Equity
ROE = stock.info['returnOnEquity']

# Payout Ratio
try:
    payout_ratio = stock.info['payoutRatio']

except:
    payout_ratio = None

# Price/Earning Ratio
PE_ttm = share_price/EPS

# Sustainable Growth Rate (not fool-proofed)
#SGR = (1-payout_ratio) * ROE

# Free Growth Estimate
FGE = 0.00

# Growth Rate (requires some thought)
GR_estimate = 0.05

cost_of_growth = GR_estimate / ROE * EPS

funding_gap = EPS  - dividend_rate - cost_of_growth

share_change = funding_gap / share_price

total_dividend_growth = GR_estimate + share_change

projected_total_return = total_dividend_growth + current_yield

In [ ]:
print(f'{date} - {stock_ticker}\n')

div_average_text()

print(f'\nDividend Yield: {current_yield*100:.2f}')
print(f'P/E Ratio (yahoof ttm): {pegRatio_yf}')
print(f'P/E Ratio (calculated ttm): {PE_ttm}')

print("---------------------------------------")

print(f"Dividend Rate ($): {dividend_rate:.2f}")
print(f"Divided by Share Price ($) {share_price:.2f}")
print(f"Current Yield (%): {current_yield*100:.2f}")

print("---------------------------------------")

print(f"Core Growth Estimate (%): {GR_estimate * 100:.2f}")
print(f"Divided by: Return on Equity (%): {ROE * 100:.2f}")
print(f"Multipled by: Earnings per Share ($): {EPS:.2f}")
print(f"Cost of Growth ($): {cost_of_growth:.2f}")

print("---------------------------------------")
print(f"Earnings per Share ($): {EPS:.2f}")
print(f"Minus: Dividend: {dividend_rate:.2f}")
print(f"Minute: Cost of Growth ($): {cost_of_growth:.2f}")
print(f"Funding Gap ($): {funding_gap:.2f}")

print("---------------------------------------")
print(f"Funding Gap ($): {funding_gap:.2f}")
print(f"Divided by: Share Price ($): {EPS:.2f}")
print(f"Share Change (%): {share_change *100:.2f}")

print("---------------------------------------")
print(f"Core Growth Estimate (%): {GR_estimate*100:.2f}")
print(f"Plus: Share Change (%): {share_change*100:.2f}")
print(f"Total Dividend Growth (%): {total_dividend_growth*100:.2f}")


print("---------------------------------------")
print(f"Plus: Dividend Yield (%): {current_yield*100:.2f}")
print(f"Projected Total Return (%): {projected_total_return*100:.2f}")



2023 - HSY

Averages           | Div Growth %  | Yield
2003-2023 (20 yrs) | 9.46 | 2.90
2012-2023 (10 yrs) | 10.08 | 2.51
2018-2023 (5 yrs)  | 9.60 | 2.10

Dividend Yield: 2.50
P/E Ratio (yahoof ttm): 20.286351500000002
P/E Ratio (calculated ttm): 15.978725017326955
---------------------------------------
Dividend Rate ($): 4.77
Divided by Share Price ($) 190.86
Current Yield (%): 2.50
---------------------------------------
Core Growth Estimate (%): 5.00
Divided by: Return on Equity (%): 54.35
Multipled by: Earnings per Share ($): 11.94
Cost of Growth ($): 1.10
---------------------------------------
Earnings per Share ($): 11.94
Minus: Dividend: 4.77
Minute: Cost of Growth ($): 1.10
Funding Gap ($): 6.08
---------------------------------------
Funding Gap ($): 6.08
Divided by: Share Price ($): 11.94
Share Change (%): 3.18
---------------------------------------
Core Growth Estimate (%): 5.00
Plus: Share Change (%): 3.18
Total Dividend Growth (%): 8.18
--------------------------------

## Margin of Safety

The basic margin of safety principle is to pay less for a stock than it is worth where:

Dividend Yield + Dividend Growth = Total Return + Margin of Safety = Hurdle Rate

refer to figure 8.3 in the Ultimate Dividend Playbook

In [ ]:
hurdle_table = pd.DataFrame({"Dividend Yield":range(1,11), "Dividend Growth": range(7,-3,-1),"Total Return":10*[8],
             "Margin of Safety":[4,3,2,1,1,1,1,2,3,4], "Hurdle Return":[12,11,10,9,9,9,9,10,11,12], "Implied Dividend Growth":[11,9,7,5,4,3,2,2,2,2]
                })

In [ ]:
hurdle_table.head(10)

,Dividend Yield,Dividend Growth,Total Return,Margin of Safety,Hurdle Return,Implied Dividend Growth
0,1,7,8,4,12,11
1,2,6,8,3,11,9
2,3,5,8,2,10,7
3,4,4,8,1,9,5
4,5,3,8,1,9,4
5,6,2,8,1,9,3
6,7,1,8,1,9,2
7,8,0,8,2,10,2
8,9,-1,8,3,11,2
9,10,-2,8,4,12,2


In [ ]:
"""
required_return - the demand of return we want from our investment. See table above to make a decision
"""
hurdle_return = .09

hurdle_price = dividend_rate / (hurdle_return - total_dividend_growth)

projected_dividend_yield = dividend_rate / hurdle_price

ratio = hurdle_price / share_price

In [ ]:
print(f"For {date} {stock_ticker}: \nDividend Rate ($): {dividend_rate:.2f}\nDividend Yield (%): {current_yield*100:.2f}\nDividend Growth (%): {total_dividend_growth*100:.2f}")

For 2023 HSY: 
Dividend Rate ($): 4.77
Dividend Yield (%): 2.50
Dividend Growth (%): 8.18


In [ ]:
print(f"To expect a {hurdle_return*100:.2f}% return from {stock_ticker}, the hold-out price is {hurdle_price:.2f}")
print(f"At a price of {hurdle_price:.2f} and a dividend rate of {dividend_rate:.2f}, the projected dividend yield is: {projected_dividend_yield *100:.2f}%.")
print(f"Hurdle Price / Current stock price is: {ratio:.2f}")

To expect a 9.00% return from HSY, the hold-out price is 584.12
At a price of 584.12 and a dividend rate of 4.77, the projected dividend yield is: 0.82%.
Hurdle Price / Current stock price is: 3.06


### Glossary

1. Return-on-Equity
    - <a href = "https://www.investopedia.com/terms/r/returnonequity.asp#:~:text=Return%20on%20equity%20(ROE)%20is%20the%20measure%20of%20a%20company's,its%20equity%20financing%20into%20profits." >Return on equity (ROE)</a> is the measure of a company's net income divided by its shareholders' equity.
    - The higher the ROE, the better a company is at converting its equity financing into profits.

### Research/References

1. Documentation
    - https://github.com/ranaroussi/yfinance
2. Articles
    - The Ultimate Dividend Playbook: Income, Insight and Independence for Today's Investor, Peters, Josh
    - https://dividendgrowthinvestingandretirement.com/estimate-dividend-growth-total-returns-using-josh-peters-dividend-drill-return-model-example-spreadsheet/#:~:text=Guessing%20a%20reasonable%20future%20dividend%20growth%20rate%20is%20hard%2C%20but,dividend%20growth%20and%20total%20returns.
    - Negative cost of growth: https://www.investopedia.com/terms/n/negative-growth.asp#:~:text=An%20economy%20with%20negative%20growth,of%20a%20recession%20or%20depression.
3. References
    - https://www.nasdaq.com/market-activity/quotes/dividend-history
4. Questions:
    - what happens if the calculated hurdle price is in the negative? that can't be a good thing? it means, the growth I am looking for is not sustainable. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=67b8993f-5b2d-4ce3-8c93-aa21b89c512f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>